# Import needed libraries

In [2]:
import pandas as pd
import numpy as np

# Load Data

In [3]:
# Load Data from CSV file
products = pd.read_csv("products.csv", sep='\t', low_memory=False)

In [4]:
# Take a look at the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(products.shape)
products.head()

(1375940, 181)


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,1569.0,1569.0,NaN,7.0,3.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.1,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Check NA values

In [5]:
print(round(products.isnull().mean().sort_values() * 100))

code                                            0.0
states_en                                       0.0
states                                          0.0
last_modified_datetime                          0.0
last_modified_t                                 0.0
states_tags                                     0.0
created_t                                       0.0
url                                             0.0
created_datetime                                0.0
creator                                         0.0
pnns_groups_2                                   0.0
countries                                       0.0
countries_tags                                  0.0
countries_en                                    0.0
pnns_groups_1                                   1.0
product_name                                    4.0
energy_100g                                    19.0
proteins_100g                                  19.0
fat_100g                                       20.0
carbohydrate

# Drop columns with a great amount of NA values

In [6]:
# Drop columns with more than 99.5% of NA values
print(products.shape)
cleanedProducts = products.dropna(axis=1, thresh=(products.shape[0]//200))
print(cleanedProducts.shape)

(1375940, 181)
(1375940, 100)


In [7]:
print(round(cleanedProducts.isnull().mean().sort_values() * 100))

code                                           0.0
url                                            0.0
created_t                                      0.0
created_datetime                               0.0
last_modified_t                                0.0
last_modified_datetime                         0.0
states_tags                                    0.0
states_en                                      0.0
states                                         0.0
creator                                        0.0
pnns_groups_2                                  0.0
countries                                      0.0
countries_en                                   0.0
countries_tags                                 0.0
pnns_groups_1                                  1.0
product_name                                   4.0
energy_100g                                   19.0
proteins_100g                                 19.0
fat_100g                                      20.0
carbohydrates_100g             

# Remove redundant columns

In [8]:
# The same column information can exist in its basic column name, and also with suffixes _tags and _en
# Keep _tags columns and remove the basic ones (_tags is normalized and contains less NA values)
# Remove _en columns if either the basic ones or _tags columns exist
columnsToRemove = []
for column in cleanedProducts.columns:
    if(column.endswith('_tags')):
#         print (column)
        columnToRemove = column.replace('_tags','')
        if columnToRemove in cleanedProducts.columns:
#             print("Column to remove: ", columnToRemove)
            columnsToRemove.append(columnToRemove)
    if(column.endswith('_en')):
#         print (column)
        columnToKeep = column.replace('_en','')
        columnToKeep_tag = column.replace('_en','_tags')
        if columnToKeep in cleanedProducts.columns:
#             print("Column to remove is: ", column)
            columnsToRemove.append(column)
        else:
            if columnToKeep_tag in cleanedProducts.columns:
#                 print("Column to remove is: ", column)
                columnsToRemove.append(column)
print("Columns to remove: ", columnsToRemove)
cleanedProducts = cleanedProducts.drop(columns=columnsToRemove)
print(cleanedProducts.shape)
cleanedProducts.head()

Columns to remove:  ['packaging', 'brands', 'categories', 'categories_en', 'origins', 'manufacturing_places', 'labels', 'labels_en', 'emb_codes', 'countries', 'countries_en', 'traces', 'traces_en', 'additives_en', 'states', 'states_en', 'main_category_en']
(1375940, 83)


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging_tags,brands_tags,categories_tags,origins_tags,manufacturing_places_tags,labels_tags,emb_codes_tags,first_packaging_code_geo,cities_tags,purchase_places,stores,countries_tags,ingredients_text,allergens,traces_tags,serving_size,serving_quantity,additives_n,additives_tags,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states_tags,brand_owner,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,potassium_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed,en:nutrition-facts-complete...",NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,1569.0,1569.0,7.0,3.08,NaN,NaN,NaN,NaN,70.1,15.0,NaN,7.8,1.4,0.560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,NaN,courte-paille,"en:groceries,en:condiments,en:sauces,en:mustards",NaN,NaN,fr:delois-france,NaN,NaN,NaN,NaN,courte paille,en:france,eau graines de téguments de moutarde vinaigre ...,en:mustard,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,18.0,d,NaN,Fat and sauces,Dressings and sauces,"en:to-be-completed,en:nutrition-facts-complete...",NaN,en:mustards,https://static.openfoodfacts.org/images/produc...,https://static.openf

# Check duplicates

In [9]:
print ('Whole row duplicates: {}'.format(cleanedProducts.duplicated().sum()))
print ('Duplicates with code: {}'.format(cleanedProducts.duplicated(subset=['code']).sum()))


Whole row duplicates: 67
Duplicates with code: 246


# Drop duplicates

In [10]:
copyCleanedProducts = cleanedProducts.copy()
print("Shape before dropping row duplicates: ", copyCleanedProducts.shape)
copyCleanedProducts = copyCleanedProducts.drop_duplicates()
print("Shape after dropping row duplicates: ", copyCleanedProducts.shape)
dups = copyCleanedProducts[copyCleanedProducts.duplicated(subset=['code'])]
print("Code duplicates shape: ", dups.shape)
dups.head()

Shape before dropping row duplicates:  (1375940, 83)
Shape after dropping row duplicates:  (1375873, 83)
Code duplicates shape:  (179, 83)


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging_tags,brands_tags,categories_tags,origins_tags,manufacturing_places_tags,labels_tags,emb_codes_tags,first_packaging_code_geo,cities_tags,purchase_places,stores,countries_tags,ingredients_text,allergens,traces_tags,serving_size,serving_quantity,additives_n,additives_tags,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states_tags,brand_owner,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,potassium_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g
24564,0016000459335,http://world-en.openfoodfacts.org/product/0016...,openfoodfacts-contributors,1483671925,2017-01-06T03:05:25Z,1587648882,2020-04-23T13:34:42Z,Gmills hny nut cheerios sweetened whl grn oat ...,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:united-states,"Whole grain oats, sugar, oat bran, corn starch...",NaN,NaN,3/4 cup (28 g) (28 g),28.0,1.0,"en:e340,en:e340iii",0.0,NaN,0.0,NaN,12.0,d,4.0,Cereals and potatoes,Cereals,"en:to-be-completed,en:nutrition-facts-complete...",GENERAL MILLS SALES INC.,en:cereals-and-their-products,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,1644.0,1644.0,5.36,0.00,1.79,1.79,0.0,0.0,78.57,32.14,7.1,7.14,1.4275,0.571,NaN,0.000536,NaN,0.0214,0.001,0.001518,NaN,0.001786,NaN,NaN,0.000005,0.411,0.357,0.286,0.01607,0.086,0.01339,NaN,NaN,NaN,12.0
30281,0019320001376,http://world-en.openfoodfacts.org/product/0019...,openfoodfacts-contributors,1374110675,2013-07-18T01:24:35Z,1587664437,2020-04-23T17:53:57Z,Kraft nabisco cookies honey 72x0.75 oz,NaN,NaN,NaN,"kraft-foods-inc,nabisco,monsanto-company","en:snacks,en:sweet-snacks,en:biscuits-and-cake...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:united-states,"Unbleached enriched flour (wheat flour, niacin...",en:gluten,NaN,1 pack (21 g) (21 g),21.0,1.0,"en:e322,en:e322i",0.0,NaN,0.0,NaN,12.0,d,4.0,Sugary snacks,Biscuits and cakes,"en:to-be-completed,en:nutrition-facts-complete...",Mondelez Int. US (0074819091009),en:biscuits,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,1795.0,1795.0,14.29,2.38,NaN,NaN,0.0,0.0,76.19,23.81,4.8,4.76,1.2500,0.500,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.143,0.429,NaN,0.00724,NaN,0.00610,NaN,NaN,NaN,12.0
128526,0051500006962,http://world-en.openfoodfacts.org/product/0051...,openfoodfacts-contributors,1416251413,2014-11-17T19:10:13Z,1542650866,2018-11-19T18:07:46Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:united-states,"STRAWBERRY JUICE, HIGH FRUCTOSE CORN SYRUP, CO...",NaN,NaN,NaN,NaN,2.0,"en:e330,en:e440,en:e440i",0.0,NaN,0.0,NaN,NaN,NaN,4.0,unknown,unknown,"en:to-be-completed,en:nutrition-facts-complete...",NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [11]:
# For each group of duplicates, keep the non NA value for each column(taken from the first duplicate where the info is 
# available) and store it in the first duplicate to keep it and drop the others
pd.set_option('display.max_colwidth', 200)
reCleanedProducts = copyCleanedProducts.copy() 
dups.reset_index()
for row in dups.iterrows():
    subDups = copyCleanedProducts[copyCleanedProducts['code'] == row[1]['code']]
    subDups = subDups.reset_index()
    ser = pd.Series(subDups.iloc[0])
    for column in dups.columns:
        idx = subDups[column].first_valid_index()
        firstIdx = idx
        if idx is not None:
            while ( (subDups[column].iloc[idx] == 'unknown') | (pd.isnull(subDups[column].iloc[idx])) ):
                if (idx <= (subDups.shape[0]-2)) :
                    idx += 1
                else: 
                    idx = firstIdx
                    break
            ser[column] = subDups[column].iloc[idx]
            reCleanedProducts.at[subDups['index'].iloc[0], column] = subDups[column].iloc[idx]

print("Shape before dropping code duplicates: ", reCleanedProducts.shape)
reCleanedProducts = reCleanedProducts.drop_duplicates(subset=['code'], keep='first')
print("Shape after dropping code duplicates: ", reCleanedProducts.shape)


Shape before dropping code duplicates:  (1375873, 83)
Shape after dropping code duplicates:  (1375694, 83)


In [12]:
# Check duplicates with the same url 
print ('Duplicates with url only: {}'.format(reCleanedProducts.duplicated(subset=['url']).sum()))

Duplicates with url only: 0


In [13]:
pd.reset_option('display.max_colwidth')
reCleanedProducts.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging_tags,brands_tags,categories_tags,origins_tags,manufacturing_places_tags,labels_tags,emb_codes_tags,first_packaging_code_geo,cities_tags,purchase_places,stores,countries_tags,ingredients_text,allergens,traces_tags,serving_size,serving_quantity,additives_n,additives_tags,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states_tags,brand_owner,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,potassium_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed,en:nutrition-facts-complete...",NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,1569.0,1569.0,7.0,3.08,NaN,NaN,NaN,NaN,70.1,15.0,NaN,7.8,1.4,0.560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,NaN,courte-paille,"en:groceries,en:condiments,en:sauces,en:mustards",NaN,NaN,fr:delois-france,NaN,NaN,NaN,NaN,courte paille,en:france,eau graines de téguments de moutarde vinaigre ...,en:mustard,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,18.0,d,NaN,Fat and sauces,Dressings and sauces,"en:to-be-completed,en:nutrition-facts-complete...",NaN,en:mustards,https://static.openfoodfacts.org/images/produc...,https://static.openf

# Remove obvious outliers and fill missing values

In [14]:
# 'fat_100g', 'saturated-fat_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 'trans-fat_100g', 
# 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g',
# 'alcohol_100g', 'vitamin-a_100g', 'vitamin-d_100g', 'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g', 'vitamin-pp_100g',
# 'vitamin-b6_100g', 'vitamin-b9_100g', 'folates_100g', 'vitamin-b12_100g', 'potassium_100g', 'calcium_100g', 'phosphorus_100g',
# 'iron_100g', 'magnesium_100g', 'zinc_100g', 'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g'

list_100g = ['fat_100g', 'saturated-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g',
'fiber_100g', 'proteins_100g', 'salt_100g']

# Pick only rows where the relevant 100g features values are indeed between 0 and 100
subset = reCleanedProducts.copy()
for it in list_100g:
    subset = subset[(subset[it] <= 100) & (subset[it] >= 0)]
    print(subset.shape)
    # Fill with mean if missing values
    if subset[it].isnull().mean() > 0:
        print(it+" missing values rate: ")
        print(str(round(subset[it].isnull().mean() * 100, 2))+"%")
        subset[it].fillna(subset[it].mean(), inplace=True)
    
an_vars = ['energy-kcal_100g', 'energy_100g', 'additives_n', 'nutrition-score-fr_100g']
for it in an_vars:
    if subset[it].isnull().mean() > 0:
        print(it+" missing values rate: ")
        print(str(round(subset[it].isnull().mean() * 100, 2))+"%")
        subset[it].fillna(subset[it].mean(), inplace=True)


(1106287, 83)
(1054641, 83)
(265383, 83)
(265248, 83)
(258485, 83)
(254233, 83)
(254048, 83)
(253642, 83)
energy-kcal_100g missing values rate: 
0.36%
energy_100g missing values rate: 
0.2%
additives_n missing values rate: 
0.49%
nutrition-score-fr_100g missing values rate: 
9.8%


# Aggregate same values in pnns_groups_1

In [15]:
print("Number of unique categories: ", subset['categories_tags'].nunique() )
print("Unique classes in categories: ",subset['categories_tags'].unique()[0:5] )
print("Number of unique pnns groups: ", subset['pnns_groups_1'].nunique() )
print("Unique classes in pnns groups: ",subset['pnns_groups_1'].unique() )

print("\nValues counts before aggregation:\n", subset['pnns_groups_1'].value_counts(normalize=True))
subset['pnns_groups_1'].replace({'salty-snacks': 'Salty snacks', 'cereals-and-potatoes': 'Cereals and potatoes',
                                  'fruits-and-vegetables': 'Fruits and vegetables', 'sugary-snacks': 'Sugary snacks'},
                                  inplace=True)
print("\nValues counts after aggregation:\n", subset['pnns_groups_1'].value_counts(normalize=True))

Number of unique categories:  3991
Unique classes in categories:  [nan
 'en:plant-based-foods-and-beverages,en:plant-based-foods,en:fruits-and-vegetables-based-foods,en:fruits-based-foods,en:fruits,en:berries,en:cranberries'
 'en:plant-based-foods-and-beverages,en:plant-based-foods,en:fruits-and-vegetables-based-foods,en:vegetables-based-foods,en:leaf-vegetables'
 'en:snacks,en:sweet-snacks,en:biscuits-and-cakes,en:viennoiseries,en:pastries,en:apple-turnovers'
 'fr:boulange']
Number of unique pnns groups:  14
Unique classes in pnns groups:  ['unknown' 'Fruits and vegetables' 'sugary-snacks' 'Composite foods'
 'Sugary snacks' 'Cereals and potatoes' 'Fat and sauces' 'Salty snacks'
 'Beverages' 'Fish Meat Eggs' 'Milk and dairy products'
 'fruits-and-vegetables' nan 'cereals-and-potatoes' 'salty-snacks']

Values counts before aggregation:
 unknown                    0.340262
Sugary snacks              0.162996
Cereals and potatoes       0.101162
Milk and dairy products    0.086567
Fat and 

# Store cleaned data in CSV file

In [16]:
subset.to_csv("cleanedProducts.csv")  # encoding='utf-8'